In [8]:
import torch
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
import numpy as np
from typing import List, Tuple
import pickle
import time
from torch.utils.data import ConcatDataset, Dataset, Subset
%matplotlib inline
plt.rcParams['figure.figsize'] = [5, 5]

In [9]:
train_dataset = CIFAR10('D:/downloads/btp/outputforpaper/CIFAR10', train=True, download=True, transform=transforms.ToTensor())
test_dataset = CIFAR10('D:/downloads/btp/outputforpaper/CIFAR10', train=False, download=True, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [10]:
total_train_size = len(train_dataset)
total_test_size = len(test_dataset)
classes = 10
input_dim = 784
num_clients = 10
rounds = 30
batch_size = 32
in_group_rounds=3
epochs_per_client =10
learning_rate = 2e-2
num_malicious_clients=5

In [11]:
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader(DataLoader):
        def __init__(self, dl, device):
            self.dl = dl
            self.device = device

        def __iter__(self):
            for batch in self.dl:
                yield to_device(batch, self.device)

        def __len__(self):
            return len(self.dl)

device = get_device()


def average_models(new_model, next, scale):
    if new_model == None:
        new_model = next
        for key in new_model:
            new_model[key] = new_model[key] * scale
    else:
        for key in new_model:
            new_model[key] = new_model[key] + (next[key] * scale)
    return new_model

In [12]:
class FederatedNet(torch.nn.Module):    
    def __init__(self):
        super().__init__()
        # Update input channels to 3 for RGB images
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 16, kernel_size=5, padding=2),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=5, padding=2),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        
        self.fc = torch.nn.Linear(8 * 8 * 32, 10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
    def batch_accuracy(self, outputs, labels):
        with torch.no_grad():
            _, predictions = torch.max(outputs, dim=1)
            return torch.tensor(torch.sum(predictions == labels).item() / len(predictions))
    
    def _process_batch(self, batch):
        images, labels = batch
        outputs = self(images)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        accuracy = self.batch_accuracy(outputs, labels)
        return (loss, accuracy)
    
    def fit(self, dataset, epochs, lr, batch_size=128, opt=torch.optim.SGD):
        dataloader = DeviceDataLoader(DataLoader(dataset, batch_size, shuffle=True), device)
        optimizer = opt(self.parameters(), lr)
        history = []
        for epoch in range(epochs):
            losses = []
            accs = []
            for batch in dataloader:
                loss, acc = self._process_batch(batch)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss.detach()
                losses.append(loss)
                accs.append(acc)
            avg_loss = torch.stack(losses).mean().item()
            avg_acc = torch.stack(accs).mean().item()
            history.append((avg_loss, avg_acc))
        return history
    
    def evaluate(self, dataset, batch_size=128):
        dataloader = DeviceDataLoader(DataLoader(dataset, batch_size), device)
        losses = []
        accs = []
        with torch.no_grad():
            for batch in dataloader:
                loss, acc = self._process_batch(batch)
                losses.append(loss)
                accs.append(acc)
        avg_loss = torch.stack(losses).mean().item()
        avg_acc = torch.stack(accs).mean().item()
        return (avg_loss, avg_acc)

In [13]:
def partition_data_dirichlet(num_clients,train_dataset,total_train_size,alpha, seed=42): 
    trainset=train_dataset
    min_required_samples_per_client = 10
    min_samples = 0
    prng = np.random.default_rng(seed)

    # get the targets
    tmp_t = trainset.targets
    if isinstance(tmp_t, list):
        tmp_t = np.array(tmp_t)
    if isinstance(tmp_t, torch.Tensor):
        tmp_t = tmp_t.numpy()
    num_classes = len(set(tmp_t))
    total_samples = len(tmp_t)
    while min_samples < min_required_samples_per_client:
        idx_clients: List[List] = [[] for _ in range(num_clients)]
        for k in range(num_classes):
            idx_k = np.where(tmp_t == k)[0]
            prng.shuffle(idx_k)
            proportions = prng.dirichlet(np.repeat(alpha, num_clients))
            proportions = np.array(
                [
                    p * (len(idx_j) < total_samples / num_clients)
                    for p, idx_j in zip(proportions, idx_clients)
                ]
            )
            proportions = proportions / proportions.sum()
            proportions = (np.cumsum(proportions) * len(idx_k)).astype(int)[:-1]
            idx_k_split = np.split(idx_k, proportions)
            idx_clients = [
                idx_j + idx.tolist() for idx_j, idx in zip(idx_clients, idx_k_split)
            ]
            min_samples = min([len(idx_j) for idx_j in idx_clients])

    trainsets_per_client = [Subset(trainset, idxs) for idxs in idx_clients]
    return trainsets_per_client

In [14]:
client_datasets = partition_data_dirichlet(num_clients,train_dataset,len(train_dataset),0.5)

In [15]:
import numpy as np


class memory:

    def __init__(self, batch_number, batch_size):
        self.batch_number = batch_number
        self.batch_size = batch_size
        self.matrix = np.zeros((batch_number, batch_size), dtype=int)
        self.add_time = 0

    def add_number(self, batch_no, true_array):
        if batch_no >= self.batch_number:
            print(f"Warning: Batch number {batch_no} out of range. Skipping this entry.")
            return
        # Ensure true_array length matches batch_size by padding or truncating as necessary
        true_array = np.pad(true_array, (0, max(0, self.batch_size - len(true_array))), mode='constant')[:self.batch_size]
        self.matrix[batch_no] += true_array


    def add(self):
        self.add_time = self.add_time + 1

    def get_ambiguous(self, number):
        temp_matrix = self.matrix
        temp_matrix = np.abs(temp_matrix - np.ones(self.matrix.shape) * self.add_time * 0.5)
        temp_array = np.reshape(temp_matrix, self.matrix.shape[0] * self.matrix.shape[1])
        temp = list(map(list, zip(range(len(temp_array)), temp_array)))
        small = sorted(temp, key=lambda x: x[1], reverse=False)
        small_array = []
        for i in range(number):
            small_array.append(small[i][0])
        return small_array

In [16]:
from torch.utils.data import DataLoader
import torch
from torch import nn
import numpy as np
# import dnn_model  # Assumed custom DNN model import
# from memory import memory  # Assumed memory module import


class KMT:
    learning_rate = 0.01
    momentum = 0.5
    batch_size = 32
    batch_number = 1000
    train_loss = []
    test_loss = []
    test_accuracy = []

    def __init__(self, dataset, index, test_loader):
        # Load training set and initialize test loader
        self.load_train_set(dataset, index)
        self.test_loader = test_loader
        self.model = FederatedNet()  # Use FederatedNet model here
        self.criterion = nn.CrossEntropyLoss()
        self.distillation_criterion = nn.KLDivLoss(reduction='batchmean')
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.learning_rate, momentum=self.momentum)
        self.memory = memory(self.batch_number, self.batch_size)

    def load_train_set(self, dataset, index):
        # Subset and DataLoader for training
        train_dataset = torch.utils.data.Subset(dataset, index.astype(int))
        self.train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

    def train(self):
        # Standard training without distillation
        self.memory.add()  # Initialize memory
        running_loss = 0
        batch_number = 0

        for images, labels in self.train_loader:
            self.optimizer.zero_grad()
            output = self.model(images)
            loss = self.criterion(output, labels)

            # Accuracy calculation for memory tracking
            log_ps = torch.log_softmax(output, dim=1)
            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = (top_class == labels.view(*top_class.shape)).numpy().squeeze().astype(int).transpose()
            self.memory.add_number(batch_number, equals)

            # Backpropagation and optimizer step
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item()
            batch_number += 1

        self.train_loss.append(running_loss / len(self.train_loader))

    def distillation_train(self, images, labels, other_soft_decision):
        # Knowledge distillation training with soft labels from other clients
        self.optimizer.zero_grad()
        output = self.model(images)
        loss1 = self.criterion(output, labels)
        loss2 = self.distillation_criterion(torch.log_softmax(output, dim=1), other_soft_decision)
        loss = loss1 + loss2

        log_ps = torch.log_softmax(output, dim=1)
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = (top_class == labels.view(*top_class.shape)).numpy().squeeze().astype(int).transpose()

        # Backpropagation and optimizer step
        loss.backward()
        self.optimizer.step()
        return equals

    def test(self):
        accuracy = 0
        loss = 0
        for images, labels in self.test_loader:
            output = self.model(images)
            log_ps = torch.log_softmax(output, dim=1)
            ps = torch.exp(log_ps)
            loss += self.criterion(log_ps, labels)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))
        self.test_loss.append((loss / len(self.test_loader)).item())
        self.test_accuracy.append((accuracy / len(self.test_loader)).item())
        return accuracy / len(self.test_loader)

    def save_model(self, filename="temp_model.pt"):
        torch.save(self.model.state_dict(), filename)

    def load_model(self, filename="temp_model.pt"):
        self.model.load_state_dict(torch.load(filename))

    def get_ambiguous_data(self, array):
        np.sort(array)
        batch_number = 0
        ambiguous_images = []
        ambiguous_labels = []
        for images, labels in self.train_loader:
            for no in array:
                if int(no / self.batch_size) == batch_number:
                    column = np.mod(no, self.batch_size)
                    ambiguous_images.append(images[column])
                    ambiguous_labels.append(labels[column])
            batch_number += 1
        ambiguous_images = torch.stack(ambiguous_images)
        ambiguous_labels = torch.stack(ambiguous_labels)
        return ambiguous_images, ambiguous_labels

    def test_data(self, images, labels):
        loss = 0
        output = self.model(images)
        log_ps = torch.log_softmax(output, dim=1)
        loss += self.criterion(log_ps, labels)
        return loss

    def test_data_accuracy(self, images, labels):
        output = self.model(images)
        log_ps = torch.log_softmax(output, dim=1)
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy = torch.mean(equals.type(torch.FloatTensor))
        return accuracy


DFL

In [17]:
class Client:
    def __init__(self, client_id, dataset, test_loader):
        self.client_id = client_id
        self.dataset = dataset
        # self.test_loader = test_loader
        self.kmt = KMT(dataset, index=np.arange(len(dataset)), test_loader=test_loader)
        self.client_data = []

    def get_dataset_size(self):
        return len(self.dataset)
    
    def get_client_id(self):
        return self.client_id

    def train(self, parameters_dict, other_soft_decisions=None):
        # Load initial parameters
        self.kmt.model.load_state_dict(parameters_dict)

        # Standard training
        self.kmt.train()

        # Knowledge distillation using soft labels from other clients (if available)
        if other_soft_decisions is not None:
            for images, labels in self.kmt.train_loader:
                if images in other_soft_decisions:
                    other_soft_decision = other_soft_decisions[images]
                    self.kmt.distillation_train(images, labels, other_soft_decision)
        
        # Return the updated model parameters
        return self.kmt.model.state_dict()

    def recieve_parameters(self, client_id, client_parameter, fraction):
        client_object = {"client_id": client_id, "client_parameter": client_parameter, "fraction": fraction}
        self.client_data.append(client_object)

    def aggregated_parameters(self):
        local_aggregated_model = None
        for i in range(len(self.client_data)):
            local_aggregated_model = average_models(local_aggregated_model, self.client_data[i]["client_parameter"], self.client_data[i]["fraction"])
        self.client_data.clear()
        return local_aggregated_model


In [18]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=200, shuffle=True)
clients = [Client(i, client_datasets[i], test_loader) for i in range(num_clients)]

In [19]:
dfl_start_time = time.time()

global_net = to_device(FederatedNet(), device)
curr_parameters = global_net.state_dict()
history_dfl = []
weights = []
traffic = []
load_model = None
total_data_size = len(train_dataset)

for i in range(rounds):
    new_model = None
    soft_decisions = {}  # Dictionary to store each client's soft decisions for distillation

    # Step 1: Each client performs training and generates soft decisions
    for client in clients:
        fraction = client.get_dataset_size() / total_data_size

        if i == 0:
            # Each client trains on its dataset, possibly using other clients' soft decisions for distillation
            client_parameters = client.train(curr_parameters, other_soft_decisions=soft_decisions if i > 0 else None)
        else:
            # Each client trains on its dataset, possibly using other clients' soft decisions for distillation
            client_parameters = client.train(client.aggregated_parameters(), other_soft_decisions=soft_decisions)
        
        # Gather soft decisions for this client's data to use in distillation
        for images, labels in client.kmt.train_loader:
            output = client.kmt.model(images)
            soft_decisions[images] = torch.softmax(output, dim=1).detach()

        # Share parameters with other clients for aggregation
        for clt in clients:
            clt.recieve_parameters(client.get_client_id(), client_parameters, fraction)

        # Aggregate models across all clients
        new_model = average_models(new_model, client_parameters, fraction)

    # Update the global model and evaluate
    load_model = new_model.copy()
    global_net.load_state_dict(new_model)
    train_loss, train_acc = global_net.evaluate(train_dataset)
    dev_loss, dev_acc = global_net.evaluate(test_dataset)
    
    # Logging
    print(f'After round {i + 1}, train_loss = {round(train_loss, 4)}, dev_loss = {round(dev_loss, 4)}, dev_acc = {round(dev_acc, 4)}')
    history_dfl.append((train_acc, dev_acc, i))
    traffic.append((num_clients) * (num_clients - 1))

# Calculate total time
dfl_end_time = time.time()
dfl_time = dfl_end_time - dfl_start_time
print(f"Total training time: {dfl_time:.2f} seconds")

# Optionally save the final global model if needed
torch.save(global_net.state_dict(), "final_global_model.pt")

TypeError: len() of unsized object

In [ ]:
plt.plot([i + 1 for i in range(len(history_dfl))], [history_dfl[i][0] for i in range(len(history_dfl))], color='r', label='train accuracy')
plt.plot([i + 1 for i in range(len(history_dfl))], [history_dfl[i][1] for i in range(len(history_dfl))], color='b', label='dev accuracy')
plt.legend()
plt.title("Rounds vs Accuracy")
plt.xlabel("Number of rounds")
plt.ylabel("Accuracy")
plt.show()

HDFL

In [ ]:
class Client:
    def __init__(self, client_id, dataset, test_loader):
        self.client_id = client_id
        self.dataset = dataset
        self.kmt = KMT(dataset, index=np.arange(len(dataset)), test_loader=test_loader)
        self.client_data=[]
        self.server_id=-1
        self.distance=random.randint(1,100)

    def set_server_id(self,client_id):
        self.client_data_server=[]
        self.server_id=client_id
    
    def get_server_id(self):
        return self.server_id

    def get_dataset_size(self):
        return len(self.dataset)
    
    def get_client_id(self):
        return self.client_id

    def train(self, parameters_dict, other_soft_decisions=None):
        # Load initial parameters
        self.kmt.model.load_state_dict(parameters_dict)

        # Standard training
        self.kmt.train()

        # Knowledge distillation using soft labels from other clients (if available)
        if other_soft_decisions is not None:
            for images, labels in self.kmt.train_loader:
                if images in other_soft_decisions:
                    other_soft_decision = other_soft_decisions[images]
                    self.kmt.distillation_train(images, labels, other_soft_decision)
        
        # Return the updated model parameters
        return self.kmt.model.state_dict()

    def recieve_parameters(self, client_id, client_parameter, fraction):
        client_object = {"client_id": client_id, "client_parameter": client_parameter, "fraction": fraction}
        self.client_data.append(client_object)

    def aggregated_parameters(self):
        local_aggregated_model = None
        for i in range(len(self.client_data)):
            local_aggregated_model = average_models(local_aggregated_model, self.client_data[i]["client_parameter"], self.client_data[i]["fraction"])
        self.client_data.clear()
        return local_aggregated_model
    
    def recieve_data_as_server(self,client_parameter,fraction):
        client_object={"client_parameter":client_parameter,"fraction":fraction}
        self.client_data_server.append(client_object)
    

    def aggregate_parameter_as_server(self):
         local_aggregated_model_as_server=None
         for data in self.client_data_server:
              local_aggregated_model_as_server=average_models(local_aggregated_model_as_server,data["client_parameter"],data["fraction"])
         self.client_data_server.clear()
         return local_aggregated_model_as_server


In [ ]:
def utility_function(client_set_1,client_set_2,data):
  client_set_1_average=None
  client_set_2_average=None
  for i in client_set_1:
    client_set_1_average=average_models(client_set_1_average,data[i],1)
  for i in client_set_2:
    client_set_2_average=average_models(client_set_2_average,data[i],1)
  model1_weights = []
  model2_weights = []
  for key, value in client_set_1_average.items():
      model1_weights.append(value.detach().cpu())

  for key, value in client_set_2_average.items():
      model2_weights.append(value.detach().cpu())
  weight1_flat = torch.cat([param.view(-1) for param in model1_weights])
  weight2_flat = torch.cat([param.view(-1) for param in model2_weights])
  cosine_sim = torch.nn.functional.cosine_similarity(weight1_flat, weight2_flat, dim=0)
  return cosine_sim.item()

In [ ]:
def generateGroupsGame(data,num_clients):
    groups=[]
    for i in range(0,num_clients):
        groups.append([i])
    while True:
        for group in groups:
            for x in group:
                no_change_flag=0
                m_i_n_group=None
                m_i_n_utility=100
                minu=0
                for g in groups:
                    if g!=group:
                        if len(g) >= 1 and len(group) > 1:
                            tempset_1=g.copy()
                            tempset_1.append(x)
                            tempset_2=group.copy()
                            tempset_2.remove(x)
                            if utility_function(tempset_1,tempset_2,data) > utility_function(g,group,data): 
                                if minu < utility_function(tempset_1,tempset_2,data):
                                    m_i_n_group=g
                                    minu=utility_function(tempset_1,tempset_2,data)        
                        elif len(group)<=1:
                            if utility_function(g,group,data) < m_i_n_utility:
                                m_i_n_group=g
                                m_i_n_utility=utility_function(g,group,data)
                if m_i_n_group is not None:  
                    group.remove(x)
                    m_i_n_group.append(x)
                    no_change_flag=1
                    if len(group) <=0 :
                        groups.remove(group)
        if no_change_flag == 0:
            break
    print("end")
    return groups

In [ ]:
def groupDataLength(groups,clients):
    group_dataset_length=[]
    for grp in groups:
        totalcount=0
        for clt in grp:
            totalcount+=clients[clt].get_dataset_size()
        group_dataset_length.append(totalcount)
    return group_dataset_length

In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=200, shuffle=True)
clients = [Client(i, client_datasets[i], test_loader) for i in range(num_clients)]

In [ ]:
global_net = to_device(FederatedNet(), device)
curr_parameters = global_net.state_dict()
data = []
for client in clients:
    paramerter=client.train(curr_parameters)
    data.append(paramerter)

groups=generateGroupsGame(data,num_clients)
print(groups)
group_dataset_length=groupDataLength(groups,clients)

In [ ]:
hdfl_start_time  =time.time()

# Initializing the global model and key variables
global_net = to_device(FederatedNet(), device)
curr_parameters = global_net.state_dict()
history_hdfl = []
weights = []
traffic = []
load_model = None
total_dataset_length = len(train_dataset)
flag = True

# Training across rounds
for i in range(rounds):

    traffic_single_sum = 0
    server = []
    
    # Step 1: Choose a random server from each group for group-level aggregation
    for grp in groups:
        traffic_single_sum += len(grp) - 1
        server_id = np.random.choice(grp, p=np.ones(len(grp)) / len(grp))
        server.append(server_id)
        clients[server_id].set_server_id(server_id)
    
    k = 0  # Group index tracker
    
    # In-group mesh decentralized learning
    for grp in groups:
        flag = True
        for j in range(in_group_rounds):
            traffic_single_sum += (len(grp) - 1) * len(grp)

            soft_decisions = {}  # Dictionary to store each client's soft decisions for distillation
            
            for clt in grp:
                fraction = clients[clt].get_dataset_size() / group_dataset_length[k]
                
                # Train using the previous aggregated parameters or the current round model
                if i == 0 and flag:
                    client_parameters = clients[clt].train(curr_parameters, other_soft_decisions=soft_decisions if j > 0 else None)
                elif j == 0:
                    client_parameters = clients[clt].train(load_model, other_soft_decisions=soft_decisions if j > 0 else None)
                else:
                    client_parameters = clients[clt].train(clients[clt].aggregated_parameters(), other_soft_decisions=soft_decisions if j > 0 else None)
                
                # Generate and store soft decisions for knowledge distillation
                soft_decisions = {}
                for images, labels in clients[clt].kmt.train_loader:
                    output = clients[clt].kmt.model(images)
                    soft_decisions[images] = torch.softmax(output, dim=1).detach()
                
                # Share soft decisions and parameters with group members
                for clt_1 in grp:
                    clients[clt_1].recieve_parameters(clt_1, client_parameters, fraction)
                    # clients[clt_1].kmt.update_soft_decisions(soft_decisions)
            
            flag = False
        
        # Aggregating on the chosen group server
        for clt in grp:
            fraction = clients[clt].get_dataset_size() / group_dataset_length[k]
            group_params = clients[clt].aggregated_parameters()
            clients[server[k]].recieve_data_as_server(group_params, fraction)
        
        k += 1
    
    # Step 2: Inter-group aggregation across group servers
    aggregateds = []
    for clt in server:
        aggregated_params = clients[clt].aggregate_parameter_as_server()
        aggregateds.append(aggregated_params)
    
    # Choose a central server among group servers for top-level aggregation
    server_id = np.random.choice(server, p=np.ones(len(server)) / len(server))
    clients[server_id].set_server_id(server_id)
    traffic_single_sum += len(server)
    
    # Aggregate parameters at the top-level server and share with the main server
    group_number = 0
    for clt in aggregateds:
        fraction = group_dataset_length[group_number] / total_dataset_length
        group_number += 1
        clients[server_id].recieve_data_as_server(clt, fraction)
    
    # Top-level aggregation on the main server
    traffic_single_sum += num_clients
    aggregated_parameters_top = clients[server_id].aggregate_parameter_as_server()
    load_model = aggregated_parameters_top.copy()
    
    # Log weights and update the global model
    total_norm = sum(torch.norm(value.view(-1)).item() for value in load_model.values())
    weights.append(total_norm)
    global_net.load_state_dict(load_model)
    
    # Evaluation after each round
    train_loss, train_acc = global_net.evaluate(train_dataset)
    dev_loss, dev_acc = global_net.evaluate(test_dataset)
    print(f'After round {i + 1}, train_loss = {round(train_loss, 4)}, dev_loss = {round(dev_loss, 4)}, dev_acc = {round(dev_acc, 4)}')
    
    traffic.append(traffic_single_sum)
    history_hdfl.append((train_acc, dev_acc, i))

# Calculate total time
hdfl_end_time = time.time()
hdfl_time = hdfl_end_time - hdfl_start_time
print(f"Total training time: {hdfl_time:.2f} seconds")

# Save the final model if needed
torch.save(global_net.state_dict(), "final_global_model.pt")

In [ ]:
plt.plot([i + 1 for i in range(len(history_hdfl))], [history_hdfl[i][0] for i in range(len(history_hdfl))], color='r', label='train accuracy')
plt.plot([i + 1 for i in range(len(history_hdfl))], [history_hdfl[i][1] for i in range(len(history_hdfl))], color='b', label='dev accuracy')
plt.legend()
plt.title("Rounds vs Accuracy")
plt.xlabel("Number of rounds")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
plt.plot([i + 1 for i in range(len(history_dfl))], [history_dfl[i][1] for i in range(len(history_dfl))], color='b', label='dev accuracy')
plt.plot([i + 1 for i in range(len(history_hdfl))], [history_hdfl[i][1] for i in range(len(history_hdfl))], color='r', label='dev accuracy')
plt.legend()
plt.title("Rounds vs Accuracy on CIFAR10")
plt.xlabel("Number of rounds")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
labels = ['DFL', 'HDFL']
values = [dfl_time, hdfl_time]

# Plot the bar graph
plt.bar(labels, values, color=['green', 'red'])

# Add labels and title
plt.xlabel('')
plt.ylabel('RUN TIME')
plt.title('Run Time for DFL & HDFL')

# Show the plot
plt.show()